In [8]:
import pandas as pd
import conf
import yaml
from friendlywater.water_stream import *
from importlib.resources import open_text
from typing import TextIO
from pandas import DataFrame


def load_csv(file: TextIO, cols) -> DataFrame:
    try:
        return pd.read_csv(file, delimiter=';', header=0,
                           names=cols,
                           dtype={
                                cols[0]: "float64",
                                cols[1]: "float64",
                                cols[2]: "float64"
                            })
    except (TypeError, ValueError) as ex:
        print("Invalid data on {filename} - Some of the columns contain strings"
              .format(filename=file.name))
        exit(1)

In [10]:
data = yaml.load(open_text(conf, 'application.yaml'), Loader=yaml.FullLoader)

water_stream = WaterStream(
    slope=data.get('stream').get('slope'),
    conveyance=data.get('stream').get('conveyance'),
    delta_t=data.get('stream').get('delta_t'),
    delta_x=data.get('stream').get('delta_x')
)
water_channel = WaterChannel(
    length_in_meters=data.get('channel').get('length'),
    height_in_meters=data.get('channel').get('height'),
    width_in_meters=data.get('channel').get('width')
)

initial = load_csv(file=open_text(conf, 'initial_conditions.csv'),
                   cols=['time', 'q0', 'h0'])

bound_left = load_csv(file=open_text(conf, 'boundaries_left.csv'),
                      cols=['time', 'q_left', 'h_left'])

bound_right = load_csv(file=open_text(conf, 'boundaries_right.csv'),
                       cols=['time', 'q_right', 'h_right'])

boundaries = pd.DataFrame({
    'time': bound_left['time'],
    'q_left': bound_left['q_left'],
    'q_right': bound_right['q_right'],
    'h_left': bound_left['h_left'],
    'h_right': bound_right['h_right']
})


In [11]:
boundaries.dtypes

time       float64
q_left     float64
q_right    float64
h_left     float64
h_right    float64
dtype: object

In [12]:
boundaries.head(10)

,time,q_left,q_right,h_left,h_right
0,0.0,2.0,0.0,NaN,1.0
1,1.0,2.0,0.0,NaN,1.0
2,2.0,4.0,0.0,NaN,1.0
3,3.0,2.0,0.0,NaN,1.0
4,4.0,2.0,0.0,NaN,1.0
5,5.0,2.0,0.0,NaN,1.0
6,6.0,4.0,0.0,NaN,1.0
7,7.0,3.0,0.0,NaN,1.0
8,8.0,3.0,0.0,NaN,1.0
9,9.0,2.0,0.0,NaN,1.0
